In [255]:
# Initialize Otter
import otter
grader = otter.Notebook("hw02A.ipynb")

# Homework 2A: Food Safety

## Cleaning and Exploring Data with Pandas

## Due Date: Thursday, Sep 7, 11:59 PM
You must submit this assignment to Gradescope by the on-time deadline, Thursday, Sep 7, 11:59pm. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted. **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance (TAs need to sleep, after all!).

Please read the instructions carefully to submit your work to both the coding and written portals of Gradescope.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your notebook.

**Collaborators**: *list collaborators here*


## This Assignment

In this homework, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and familiarise yourself with some of the pandas functions discussed in Pandas I and II. 


After this homework, you should be comfortable with:
* Reading CSV files, 
* Reading `Pandas` documentation and using `Pandas`,
* Working with data at different levels of granularity,
* Identifying the type of data collected, missing values, anomalies, etc., and doing some basic analysis

## Score Breakdown 
Question | Manual | Points
--- | --- | ---
1a | no | 1
1b | no | 2
1c | no | 1
2a | no | 1
2bi | no | 1
2bii | no | 1
2ci | no | 1
2cii | no | 1
2d | yes | 2
2e | no | 2
3a | no | 1
3b | no | 2
3ci | no | 1
3cii | no | 1
3ciii | no | 1
3civ | no | 1
4a | no | 1
4b | no | 1
4c | no | 1
4d | no | 2
4e | no | 1
Total | 1 | 24


## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run codes, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `Pandas` and `Numpy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.

The cell below imports all the necessary libraries you need to use during this homework. Without running this cell, you will not be able to call the various numpy and pandas functions we use later on, so please make sure you run it before starting to work on the homework.

In [256]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')

from IPython.display import display, Image 
def display_figure_for_grader(fig):
    plotly.io.write_image(fig, 'temp.png')
    display(Image('temp.png'))    

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# 0: Obtaining the Data

## File Systems and I/O


In general, we will focus on using Python commands to investigate files.  However, it can sometimes be easier to use shell commands in your local operating system.  The following cells demonstrate how to do this.

In [257]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('data.zip')
dest_path = file_path

After running the cell above, if you list the contents of the directory containing this notebook, you should see `data.zip`.

*Note*: The command below starts with an `!`. This tells our Jupyter Notebook to pass this command to the operating system. In this case, the command is the `ls` Unix command which lists files in the current directory.

In [258]:
!dir

 Volume in drive F is F
 Volume Serial Number is 0810-0596

 Directory of f:\TYZ\Projects_abord\Data100\hw\hw02A

2023/12/23  11:05    <DIR>          .
2023/12/23  11:05    <DIR>          ..
2023/12/23  15:08            49,928 .OTTER_LOG
2023/12/23  11:05    <DIR>          data
2023/12/22  13:06           609,879 data.zip
2023/12/23  15:05           138,886 hw02A.ipynb
               3 File(s)        798,693 bytes
               3 Dir(s)  1,431,187,791,872 bytes free


## Loading Food Safety Data

We have data, but we don't have any specific questions about the data yet. Let's focus on understanding the structure of the data; this involves answering questions such as:

* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record? (We sometimes also use the term 'feature' or 'attribute' as well, depending on the context)

Let's start by looking at the contents of `data.zip`. It's not just a single file but rather a compressed directory of multiple files. We could inspect it by uncompressing it using a shell command such as `!unzip data.zip`, but in this homework we're going to do almost everything in Python for maximum portability.

## Looking Inside and Extracting the Zip Files

The following code blocks are for setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.

Here, we assign `my_zip` to a `zipfile.Zipfile` object representing `data.zip`, and assign `list_names` to a list of all the names of the contents in `data.zip`.

In [259]:
import zipfile
my_zip = zipfile.ZipFile(dest_path, 'r')
list_names = my_zip.namelist()
list_names

['data/',
 'data/bus.csv',
 'data/ins.csv',
 'data/ins2vio.csv',
 'data/vio.csv',
 'data/sf_zipcodes.json',
 'data/legend.csv']

You may notice that we did not write `zipfile.ZipFile('data.zip', ...)`. Instead, we used `zipfile.ZipFile(dest_path, ...)`. In general, we **strongly suggest having your filenames hard coded as string literals only once** in a notebook. It is very dangerous to hardcode things twice because if you change one but forget to change the other, you can end up with bugs that are very hard to find.

Now, we display the files' names and their sizes.

In [260]:
my_zip = zipfile.ZipFile(dest_path, 'r')
for info in my_zip.infolist():
    print('{}\t{}'.format(info.filename, info.file_size))

data/	0
data/bus.csv	665365
data/ins.csv	1860919
data/ins2vio.csv	1032799
data/vio.csv	4213
data/sf_zipcodes.json	474
data/legend.csv	120


Often when working with zipped data, we'll never unzip the actual zipfile. This saves space on our local computer. However, for this homework the files are small, so we're just going to unzip everything. This has the added benefit that you can look inside the CSV files using a text editor, which might be handy for understanding the structure of the files. The cell below will unzip the CSV files into a sub-directory called `data`.

In [261]:
data_dir = Path('.')
my_zip.extractall(data_dir)
!dir {data_dir / Path("data")}

 Volume in drive F is F
 Volume Serial Number is 0810-0596

 Directory of f:\TYZ\Projects_abord\Data100\hw\hw02A\data

2023/12/23  11:05    <DIR>          .
2023/12/23  11:05    <DIR>          ..
2023/12/23  15:08           665,365 bus.csv
2023/12/23  15:08         1,860,919 ins.csv
2023/12/23  15:08         1,032,799 ins2vio.csv
2023/12/23  15:08               120 legend.csv
2023/12/23  15:08               474 sf_zipcodes.json
2023/12/23  15:08             4,213 vio.csv
               6 File(s)      3,563,890 bytes
               2 Dir(s)  1,431,187,791,872 bytes free


The cell above created a folder called `data`, and in it there should be five CSV files. Let's open up `legend.csv` to see its contents. To do this, click on the file icon on the top left to show the folders and files within the hw02A folder, then click on `legend.csv`. The file will open up in another tab. You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

The `legend.csv` file does indeed look like a well-formed CSV file. Let's check the other three files. Rather than opening up each file manually, let's use Python to print out the first 5 lines of each. We defined a helper function for you that will allow you to retrieve the first N lines of a file as a list. For example, `head('data/legend.csv', 5)` will return the first 5 lines of "data/legend.csv". Run the cell below to print out the first 5 lines of all six files that we just extracted from the zipfile.

In [262]:
import os

def head(filename, lines=5):
    """
    Returns the first few lines of a file.
    
    filename: the name of the file to open
    lines: the number of lines to include
    
    return: A list of the first few lines from the file.
    """
    from itertools import islice
    with open(filename, "r") as f:
        return list(islice(f, lines))

data_dir = "./"
for f in list_names:
    if not os.path.isdir(f):
        print(head(data_dir + f, 5), "\n")

['"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n', '"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n', '"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n', '"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n', '"100026","LOCAL CATERING","1566 CARROLL AVE","San Francisco","CA","94124","-9999","-9999","+14155860315"\n'] 

['"iid","date","score","type"\n', '"100010_20190329","03/29/2019 12:00:00 AM","-1","New Construction"\n', '"100010_20190403","04/03/2019 12:00:00 AM","100","Routine - Unscheduled"\n', '"100017_20190417","04/17/2019 12:00:00 AM","-1","New Ownership"\n', '"100017_20190816","08/16/2019 12:00:00 AM","91","Routine - Unscheduled"\n'] 

['"iid","vid"\n', '"97975_20190725","103124"\n', '"85986_20161011","103114"\n', '"95754_2019032

## Reading in and Verifying Data

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `ins.csv`, and `vio.csv` into Pandas Dataframes with the following names: `bus`, `ins2vio`, `ins`, and `vio`, respectively.

*Note:* Because of character encoding issues, one of the files (`bus`) will require an additional argument `encoding='ISO-8859-1'` when calling `pd.read_csv`. At some point in your future, you should read all about [character encodings](https://diveintopython3.problemsolving.io/strings.html). We won't discuss these in detail in Data 100.

In [263]:
# Path to the directory containing data
dsDir = Path('data')

bus = pd.read_csv(dsDir/'bus.csv', encoding='ISO-8859-1')
ins2vio = pd.read_csv(dsDir/'ins2vio.csv')
ins = pd.read_csv(dsDir/'ins.csv')
vio = pd.read_csv(dsDir/'vio.csv')

# This code is essential for the autograder to function properly. Do not edit
ins_test = ins

Now that you've read the files, let's try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/pandas-docs/version/1.4.3/reference/api/pandas.DataFrame.html)).
Use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, and `vio` Dataframes. For example, running the cell below will display the first few lines of the `bus` Dataframe. 

In [264]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


To show multiple return outputs in one single cell, you can use `display()`. 

In [265]:
display(bus.head())
display(ins.head())

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


The `DataFrame.describe` method can also be handy for computing summaries of numeric columns of our Dataframes. Try it out with each of our 4 Dataframes. Below, we have used the method to give a summary of the `bus` Dataframe. 

In [266]:
bus.describe()

,business id column,latitude,longitude,phone_number
count,6253.000000,6253.000000,6253.000000,6.253000e+03
mean,60448.948984,-5575.337966,-5645.817699,4.701819e+09
std,36480.132445,4983.390142,4903.993683,6.667508e+09
min,19.000000,-9999.000000,-9999.000000,-9.999000e+03
25%,18399.000000,-9999.000000,-9999.000000,-9.999000e+03
50%,75685.000000,-9999.000000,-9999.000000,-9.999000e+03
75%,90886.000000,37.776494,-122.421553,1.415533e+10
max,102705.000000,37.824494,0.000000,1.415988e+10


Now, we perform some sanity checks for you to verify that the data was loaded with the correct structure.

First, we check the basic structure of the data frames you created:

In [267]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [268]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses Dataframe:')
display(bus_summary)
print('What we expect from your Inspections Dataframe:')
display(ins_summary)
print('What we expect from your Violations Dataframe:')
display(vio_summary)

What we expect from your Businesses Dataframe:


,business id column,latitude,longitude
min,19.0,-9999.000000,-9999.0
50%,75685.0,-9999.000000,-9999.0
max,102705.0,37.824494,0.0


What we expect from your Inspections Dataframe:


,score
min,-1.0
50%,76.0
max,100.0


What we expect from your Violations Dataframe:


,vid
min,103102.0
50%,103135.0
max,103177.0


The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your Dataframe are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding errors can cause spurious failures. Run the following cells to load some basic utilities (you do not need to change these at all):

In [269]:
"""Run this cell to load this utility comparison function that we will use in various
tests below (both tests you can see and those we run internally for grading).

Do not modify the function in any way.
"""


def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two Dataframes on a few summary statistics.
    
    Compute the min, median and max of the two Dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.
    
    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """    
    # Summary statistics to compare on
    stats = ['min', '50%', 'max']
    
    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if column is None. In that case, 
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` Dataframe.

<br/><br/><br/>

---

<br/><br/><br/>

# Question 1: Examining the Business Data File

## Question 1a

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [270]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


The `bus` Dataframe contains a column called `business id column` which probably corresponds to a unique business id.  However, we will first rename that column to `bid` for simplicity.

**Note**: In practice, we might want to do this renaming when the table is loaded but for grading purposes we will do it here.


In [271]:
bus = bus.rename(columns={"business id column": "bid"})

Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

**Hint**: Use [`value_counts()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) or [`unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) to determine if the `bid` series has any duplicates.

In [272]:
is_bid_unique = bus.value_counts("bid").shape[0] == bus.shape[0]
is_bid_unique

True

In [273]:
grader.check("q1a")

q1a results: All test cases passed!

<br/><br/><br/>

---


## Question 1b

We will now work with some important fields in `bus`.

1. Assign `top_names` to an iterable containing the top 5 most frequently used business names, from most frequent to least frequent. 
2. Assign `top_addresses` to an iterable containing the top 5 addresses where businesses are located, from most popular to least popular.
   
Recall from CS88 or CS61A that "an iterable object is anything that can be passed to the built-in iter function. Iterables include sequence values such as strings and tuples, and other containers such as sets and dictionaries."

**Hint 1**: You may find [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) helpful.

**Hint 2**: You'll need to get the names / addresses, NOT the counts associated with each. Some way to **reset the index** would come in handy. If you're unsure how to do this, try looking through the class notes or using a search engine. Part of the goal of this course is to develop independent thinking in the context of the data science lifecycle, which can involve a fair bit of exploring and reading documentation. It may be a bit annoying at first, but you'll get the hang of it, and we're here to guide you on that path! 

**Hint 3**: To check your answer, `top_names[0]` should return the string `Peet's Coffee & Tea`. It should not be a number.


In [274]:
top_names = bus.value_counts("name").head(5).index
top_addresses = bus.value_counts("address").head(5).index 

display(top_names)
display(top_addresses)

Index(['Peet's Coffee & Tea', 'Starbucks Coffee', 'McDonald's', 'Jamba Juice',
       'Proper Food'],
      dtype='object', name='name')

Index(['Off The Grid', '428 11th St', '3251 20th Ave', '2948 Folsom St',
       'Pier 41'],
      dtype='object', name='address')

In [275]:
grader.check("q1b")

q1b results: All test cases passed!

<br/><br/><br/>

---


## Question 1c

Based on the above exploration, what does each record represent?

**A**. One location of a restaurant.

**B**. A chain of restaurants.

**C**. A city block.

Answer in the following cell. Your answer should be a string, either `"A"`, `"B"`, or `"C"`.


In [276]:
# What does each record represent?  Valid answers are:
#    "A"
#    "B"
#    "C"
q1c = "A"

In [277]:
grader.check("q1c")

q1c results: All test cases passed!

<br/><br/><br/>

---

<br/><br/><br/>

# 2: Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city. Let's examine and clean the postal code field. The postal code (sometimes also called a [ZIP code](https://en.wikipedia.org/wiki/ZIP_Code)) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/><br/>

---


## Question 2a

How many restaurants are in each ZIP code? 

In the cell below, create a **series** where the index is the postal code and the value is the number of records with that postal code. The series should be in descending order of count. Do you notice any odd/invalid zip codes?


In [278]:
zip_counts = bus.value_counts("postal_code")
print(zip_counts.to_string()), type(zip_counts)

postal_code
94103         562
94110         555
94102         456
94107         408
94133         398
94109         382
94111         259
94122         255
94105         249
94118         231
94115         230
94108         229
94124         218
94114         200
-9999         194
94112         192
94117         189
94123         177
94121         157
94104         142
94132         132
94116          97
94158          90
94134          82
94127          67
94131          49
94130           8
94143           5
94101           2
CA              2
94188           2
94301           2
94013           2
94518           1
95133           1
95132           1
94602           1
94544           1
95122           1
94621           1
95105           1
95109           1
95112           1
95117           1
94901           1
94105-2907      1
64110           1
94120           1
941102019       1
941033148       1
94102-5917      1
941             1
94117-3504      1
94080           1
00000           

(None, pandas.core.series.Series)

In [279]:
grader.check("q2a")

q2a results: All test cases passed!

<br/><br/><br/>

--- 

## Question 2b

In question 2a we noticed a large number of potentially invalid ZIP codes (e.g., "Ca").  These are likely due to data entry errors.  To get a better understanding of the potential errors in the zip codes, let's break down the problem into two parts.

### Part I

Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json`, and store them as a Series in `valid_zips`. As you may expect, `pd.read_json` works similarly to `pd.read_csv` but for JSON files (a different file format you'll learn more about in HW 3) that you can read more about [here](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html). If you are unsure of what data type a variable is, remember you can do `type(some_var_name)` to check!

In [280]:
zipcode_file = Path('./data') / 'sf_zipcodes.json'
# Use pd.read_json(zipcode_file, typ='series').
# Then dtype will be object.
valid_zips = pd.read_json(zipcode_file).squeeze()
valid_zips, type(valid_zips)

(0     94102
 1     94103
 2     94104
 3     94105
 4     94107
 5     94108
 6     94109
 7     94110
 8     94111
 9     94112
 10    94114
 11    94115
 12    94116
 13    94117
 14    94118
 15    94119
 16    94120
 17    94121
 18    94122
 19    94123
 20    94124
 21    94125
 22    94126
 23    94127
 24    94128
 25    94129
 26    94130
 27    94131
 28    94132
 29    94133
 30    94134
 31    94137
 32    94139
 33    94140
 34    94141
 35    94142
 36    94143
 37    94144
 38    94145
 39    94146
 40    94147
 41    94151
 42    94158
 43    94159
 44    94160
 45    94161
 46    94163
 47    94164
 48    94172
 49    94177
 50    94188
 Name: zip_codes, dtype: int64,
 pandas.core.series.Series)

In [281]:
grader.check("q2bi")

q2bi results: All test cases passed!

Observe that `pd.read_json` reads data as integers by default. This isn't quite what we want! We would like to store zip codes as strings (you'll learn more about why soon!). To do that, we can use the `astype` function to generate a copy of the Pandas `Series` stored as strings instead.

In [282]:
valid_zips = valid_zips.astype("string")

If you're ever unsure about the data type of a variable, remember you can always check using the `type` function like below:

In [283]:
type(valid_zips.dtype)

pandas.core.arrays.string_.StringDtype

Now it's time to do part II. You will probably want to use the `Series.isin` function. For more information on this function see the [the documentation linked in this internet search](https://www.google.com/search?q=series+isin+pandas&rlz=1C1CHBF_enUS910US910&oq=series+isin+pandas&aqs=chrome..69i57l2j69i59j69i60l2j69i65j69i60l2.1252j0j7&sourceid=chrome&ie=UTF-8). 

**Note:** You are welcome and, in fact, encouraged to search and read documentation on the internet to complete the assignments in the course, even if the documentation is not linked explicitly.

### Part II

 Construct a `DataFrame` containing only the businesses which DO NOT have valid ZIP codes.

In [284]:
# invalid_zip_bus = bus[]
# invalid_zip_bus.head(20)
invalid_zip_bus = bus[~bus["postal_code"].isin(valid_zips)]
invalid_zip_bus

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
68,100417,"COMPASS ONE, LLC",1 MARKET ST. FL,San Francisco,CA,94105-1420,-9999.0,-9999.0,14154324000
96,100660,TEAPENTER,1518 IRVING ST,San Francisco,CA,94122-1909,-9999.0,-9999.0,14155868318
109,100781,LE CAFE DU SOLEIL,200 FILLMORE ST,San Francisco,CA,94117-3504,-9999.0,-9999.0,14155614215
144,101084,Deli North 200,1 Warriors Way Level 300 North East,San Francisco,CA,94518,-9999.0,-9999.0,-9999
...,...,...,...,...,...,...,...,...,...
6173,99369,HOTEL BIRON,45 ROSE ST,San Francisco,CA,94102-5917,-9999.0,-9999.0,14155700403
6174,99376,Mashallah Halal Food truck Ind,Off The Grid,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
6199,99536,FAITH SANDWICH #2,560 MISSION ST,San Francisco,CA,94105-2907,-9999.0,-9999.0,14155256783
6204,99681,Twister,660 East Gish Rd,San Francisco,CA,95112,-9999.0,-9999.0,-9999


In [285]:
grader.check("q2bii")

q2bii results: All test cases passed!

<br/><br/><br/>

--- 

## Question 2c

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  Do they all share a potentially "interesting address"? For that purpose, in the following cells, we will construct a series that counts the number of businesses at each `address` that have this single likely MISSING postal code value. 

Let's break this down into steps: 

### Part 1
Identify the single common missing postal code and assign it to `missing_postal_code`. Then create a Dataframe, `bus_missing`, to store only those businesses in `bus` that have `missing_postal_code` as their postal code.

In [286]:
missing_postal_code = invalid_zip_bus.value_counts("postal_code").index[0]
bus_missing = bus[bus["postal_code"] == missing_postal_code]
missing_postal_code, bus_missing

('-9999',
          bid                            name  \
 22    100126       Lamas Peruvian Food Truck   
 156   101129                 Vendor Room 200   
 177   101192                    Cochinita #2   
 276   102014    DROPBOX (Section 3, Floor 7)   
 295   102245      Vessell CA Operations (#4)   
 ...      ...                             ...   
 5820   96774     DANIEL SCOTT RIPLEY CATERER   
 5843   96938                      POKE REESE   
 5940   97700    THE ITALIAN HOMEMADE COMPANY   
 6078   98788                       333 Truck   
 6174   99376  Mashallah Halal Food truck Ind   
 
                                   address           city state postal_code  \
 22                       Private Location  San Francisco    CA       -9999   
 156   1 Warriors Way Level 300 South West  San Francisco    CA       -9999   
 177              2 Marina Blvd Fort Mason  San Francisco    CA       -9999   
 276                         1800 Owens St  San Francisco    CA       -9999   
 295 

In [287]:
grader.check("q2ci")

q2ci results: All test cases passed!

### Part 2
Using `bus_missing`, find the number of businesses at each address (which would all share the same postal code). Specifically, `missing_zip_address_count` should store a Series with addresses as the indices and the counts as the values.

In [288]:
missing_zip_address_count = bus_missing["address"].value_counts()
missing_zip_address_count.head()

address
Off The Grid                  39
Off the Grid                  10
OTG                            4
Approved Locations             3
Approved Private Locations     3
Name: count, dtype: int64

In [289]:
grader.check("q2cii")

q2cii results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/><br/><br/>

--- 

## Question 2d

If we were to drop businesses with postal code values equal to `missing_postal_code`, what specific types of businesses would we be excluding? In other words, is there a commonality among businesses with missing postal codes?

**Hint**: You may want to identify and Google the names of the businesses with missing postal codes. Feel free to reuse parts of your code from 2c to re-examine `bus_missing`, but we will not be grading your code.


# _Type your answer here, replacing this text._
Hot Dog.

In [290]:
# Find the most popular restaurant.
bus_missing.value_counts('name')


name
Annie's Hot Dogs & Pretzels    5
Mateo's Bay Area Hot Dogs      2
Stanley's Steamers             2
Taqueria Angelica's            2
Judie's Tacos Locos            2
                              ..
Gaga's Rollin Diner            1
Garden Creamery                1
Ghirardelli Chocolate          1
Good Frikin Chicken            1
Yoshi                          1
Name: count, Length: 183, dtype: int64

<!-- END QUESTION -->

<br/><br/><br/>

--- 

## Question 2e

Examine the `invalid_zip_bus` Dataframe we computed in question 2c and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9 digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` Dataframe which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Francisco ZIP Code (according to `valid_zips`), the provided code will set the `postal5` value to `None`. 

**Hint:** You will find `str` accessors particularly useful. They allow you to use your usual Python string functions in tandem with a Dataframe. For example, if you wanted to use the `replace` function on every entry in a column of a Dataframe to change the letter 'a' to 'e', you could do so by writing `df['col_name'].str.replace('a', 'e')`. Think about the different ways you can extract the first 5 digits using regular Python code!

**Do not modify the provided code! Simply add your own code in place of the ellipses.**


In [291]:
bus['postal5'] = None
invalid_zip_bus = invalid_zip_bus[invalid_zip_bus["postal_code"] != missing_postal_code]

bus['postal5'] = bus['postal_code'].str[:5]
bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None
# Checking the corrected postal5 column
display(bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']])

,bid,name,postal_code,postal5
68,100417,"COMPASS ONE, LLC",94105-1420,94105
96,100660,TEAPENTER,94122-1909,94122
109,100781,LE CAFE DU SOLEIL,94117-3504,94117
144,101084,Deli North 200,94518,None
1365,36547,Epic Roasthouse,95105,None
1408,37167,InterContinental San Francisco Employee Cafeteria,94013,None
1410,37169,InterContinental San Francisco 4th Fl. Kitchen,94013,None
1822,5200,LATINO CUISINE CULINARY ACADEMY,941102019,94110
1824,5208,GOLDEN GATE YACHT CLUB,941,None
2135,6029,ALEXIS APARTMENTS SR. MEALS PROGRAM,941033148,94103


In [292]:
grader.check("q2e")

q2e results: All test cases passed!

<br/><br/><br/>

---

<br/><br/><br/>

# 3: Investigate the Inspection Data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [293]:
ins.head(5)

,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


<br/><br/><br/>

---

## Question 3a

The column `iid` probably corresponds to an inspection id.  Write an expression (line of code) that evaluates to `True` or `False` based on whether all the inspection ids are unique.

**Hint:** This is a very similar question to Question 1a.

In [294]:
is_ins_iid_unique = ins['iid'].unique().shape[0] == ins.shape[0]
is_ins_iid_unique 

True

In [295]:
grader.check("q3a")

q3a results: All test cases passed!

<br/><br/>

---

## Question 3b

We would like to extract `bid` from each row of the `ins` Dataframe. If we look carefully, the column `iid` of the `ins` DataFrame appears to be the composition of two numbers and the first number looks like a business id.  

Create a new column called `bid` in the `ins` Dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this. (Python's in-built `split` method could come in use, read up on the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)!) Also be sure to convert the type of this column to `int`. 

**Hint**: Similar to an earlier problem where we used `astype("string")` to convert a column to a String, here you should use `astype` to convert the `bid` column into type `int`. **No Python `for` loops or list comprehensions are allowed.** This is on the honor system since our autograder isn't smart enough to check, but if you're using `for` loops or list comprehensions, you're doing the HW incorrectly. 

In [296]:
ins['bid'] = ins['iid'].str.split('_').str[0]
ins['bid'] = ins['bid'].astype("int")
ins.head(5)

,iid,date,score,type,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017


In [297]:
grader.check("q3b")

q3b results: All test cases passed!

<br/><br/><br/>

---

## Question 3c

For this part, we're going to explore some new somewhat strange syntax that we haven't seen in lecture. Don't panic! If you're not sure what to do, try experimenting, Googling, and don't shy away from talking to other students or course staff.

For this problem we'll use the time component of the inspection data.  All of this information is given in the `date` column of the `ins` Dataframe. 

**No Python `for` loops or list comprehensions are allowed!**

### Part I

What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in Python. 

In [298]:
ins_date_type = type(ins['date'][0])
ins_date_type

str

In [299]:
grader.check("q3ci")

q3ci results: All test cases passed!

### Part II
Rather than the type you discovered in Part 1, we want each entry in `pd.TimeStamp` format. You might expect that the usual way to convert something from it current type to `TimeStamp` would be to use `astype`. You can do that, but the more typical way is to use `pd.to_datetime`. Using `pd.to_datetime`, create a new `ins['timestamp']` column containing `pd.Timestamp` objects.  These will allow us to do date manipulation with much greater ease in part III and part IV. 

Note: You may run into a UserWarning in case you do not specify the date format when using `pd.to_datetime`. To resolve this, consider using the following string '%m/%d/%Y %I:%M:%S %p' to specify the `format`.

In [300]:
ins['timestamp'] = pd.to_datetime(ins['date'], format= '%m/%d/%Y %I:%M:%S %p')
ins['timestamp'] 

0       2019-03-29
1       2019-04-03
2       2019-04-17
3       2019-08-16
4       2019-08-26
           ...    
26658   2018-09-24
26659   2018-11-02
26660   2019-09-09
26661   2017-12-07
26662   2018-08-08
Name: timestamp, Length: 26663, dtype: datetime64[ns]

In [301]:
grader.check("q3cii")

q3cii results: All test cases passed!

### Part III

What are the earliest and latest dates in our inspection data?  

**Hint**: you can use `min` and `max` on dates of the correct type.

In [302]:
earliest_date = ins['timestamp'].min()
latest_date = ins['timestamp'].max()
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2016-10-04 00:00:00
Latest Date: 2019-11-28 00:00:00


In [303]:
grader.check("q3ciii")

q3ciii results: All test cases passed!

### Part IV

We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

In case you're curious, the documentation for `TimeStamp` data can be found at [this link](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp).

In [304]:
ins['year'] = ins['timestamp'].dt.year

In [305]:
grader.check("q3civ")

q3civ results: All test cases passed!

<br/><br/><br/>

---

<br/><br/><br/>

# 4: Some Analysis

Let's try and figure out whether there are any differences between names of restaurants located in even and odd zipcodes (specifically using the 5-digit postal codes). We will break down this analysis into steps with the end goal of figuring out the restaurant with the longest name and a valid phone number, among all the even zipcodes and odd zipcodes respectively.

## Question 4a

First, create a new column `name_length` that stores the length of the `name` of each of the restaurants in `bus`. Again, **do not use for loops or list comprehensions**.

In [322]:
bus['name_length'] = bus['name'].str.len()
bus

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,21
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133,21
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103,27
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124,14
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124,16
...,...,...,...,...,...,...,...,...,...,...,...
6248,99948,SUSIECAKES BAKERY,3509 CALIFORNIA ST,San Francisco,CA,94118,-9999.000000,-9999.000000,14150452253,94118,17
6249,99988,HINODEYA SOMA,303 02nd ST STE 102,San Francisco,CA,94107,-9999.000000,-9999.000000,-9999,94107,13
6250,99991,TON TON,422 GEARY ST,San Francisco,CA,94102,-9999.000000,-9999.000000,14155531280,94102,7
6251,99992,URBAN EXPRESS KITCHENS LLC,475 06th ST,San Francisco,CA,94103,-9999.000000,-9999.000000,14150368085,94103,26


In [323]:
grader.check("q4a")

q4a results: All test cases passed!

<br/><br/>

---

## Question 4b

In order to work the 5-digit zipcodes and check whether they are even or odd, we need to ensure that there are no None values contained. Create a new Dataframe `bus_valid` which only contains rows with `postal5` values that are not None. You may find the `.isna()` function useful! For the rest of this question, we will be working with `bus_valid`.

In [324]:
help(pd.isna)

Help on function isna in module pandas.core.dtypes.missing:

isna(obj: 'object') -> 'bool | npt.NDArray[np.bool_] | NDFrame'
    Detect missing values for an array-like object.
    
    This function takes a scalar or array-like object and indicates
    whether values are missing (``NaN`` in numeric arrays, ``None`` or ``NaN``
    in object arrays, ``NaT`` in datetimelike).
    
    Parameters
    ----------
    obj : scalar or array-like
        Object to check for null or missing values.
    
    Returns
    -------
    bool or array-like of bool
        For scalar input, returns a scalar boolean.
        For array input, returns an array of boolean indicating whether each
        corresponding element is missing.
    
    See Also
    --------
    notna : Boolean inverse of pandas.isna.
    Series.isna : Detect missing values in a Series.
    DataFrame.isna : Detect missing values in a DataFrame.
    Index.isna : Detect missing values in an Index.
    
    Examples
    --------
    

In [325]:
bus_valid = bus[~bus['postal5'].isna()]
bus_valid.head()

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,21
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133,21
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103,27
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124,14
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124,16


In [326]:
grader.check("q4b")

q4b results: All test cases passed!

<br/><br/>

---

## Question 4c

Now, assign `is_even` to a boolean **series** that indicates whether the corresponding 5-digit zipcode in `bus_valid` is even or odd. Remember to keep in mind the data type of `postal5`!

Hint: You might find the mod operation `%` useful here!

In [327]:
is_even = bus_valid['postal5'].astype("int") % 2 == 0

In [328]:
grader.check("q4c")

q4c results: All test cases passed!

<br/><br/>

---

## Question 4d 

Using the series you created above, store the name of the business with the longest name amongst all businesses located in even zipcodes in `longest_name_even`. You do not have to use the skeleton code provided and can use more/fewer lines than provided, but make sure that `longest_name_even` contains a **string** with your answer. 

In [335]:
longest_name_even = bus_valid[is_even].sort_values("name_length").iloc[-1, 1]
longest_name_even

'Zaida T. Rodriguez (ZTR) Annex Child Development Center'

In [336]:
grader.check("q4d")

q4d results: All test cases passed!

<br/><br/>

---

## Question 4e 

Suppose we only consider businesses in `bus_valid` that also have valid phone numbers. That is, we no longer include businesses that have invalid phone numbers. Does your answer from 4d change? Here, an invalid phone number refers to the single common missing value for phone numbers, similar to the `missing_postal_code` you found in 2b. 

Write an expression that indicates whether the longest name is the same (True) or not (False) if we require that the business must have a valid phone number. Feel free to use the scratch cell below if need be! You may find your code from the previous part to be a useful starting point. 

In [340]:
# SCRATCH CELL
# Feel free to do your rough work here
# Do not add a cell between your solution and the grader cell
missing_number = bus["phone_number"].value_counts().index[0]

-9999

In [344]:
same_longest_name = bus_valid[is_even].sort_values("name_length").iloc[-1, -3] == missing_number
same_longest_name

False

In [345]:
grader.check("q4e")

q4e results: All test cases passed!

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Homework 2A! ##

### Submission Instructions

Below, you will see two cells.
* Running the very last cell will automatically generate a zip with your autograded answers.
* Only after you run the very last cell, you can press button on the second last cell to automatically generate a PDF of all manually graded questions in Homework 2A. Clicking on the link should open up the pdf in Jupyer and allow you to download it from there. If the link does not work, please check `File Browser` tab on the left panel and download hw02A.pdf from there.

To receive credit on this assignment, **you must submit both your coding and written portions to their respective Gradescope portals.** Your written submission (a single
PDF) can be generated as follows:

1. Access your answers to manually graded Homework 2A  questions in one of three ways:
      * <b>*Automatically create PDF (recommended)*</b>: Run the first cell below and download the generated PDF. This function will extract your response to the manually graded questions and put them on a separate page. This process may fail if your answer is not properly formatted; if this is the case, check out common errors and solution described on Ed or follow either of the two ways described below.
      * *Manually download PDF*: If there are issues with automatically generating the PDF in the first cell, you can try downloading the notebook as a PDF by clicking on `File -> Save and Export Notebook As... -> PDF`. If you choose to go this route, you must take special care to ensure all appropriate pages are chosen for each question on Gradescope.
      * *Take screenshots*: If that doesn't work either, you can take screenshots of your answers (and your code if present) to manually graded questions and include them as images in a PDF. The manually graded questions are listed at the top of the Homework 2A notebook.
    
1. **Important**: When submitting on Gradescope, you **must tag pages to each question correctly** (it prompts you to do this after submitting your work). This significantly streamlines the grading process for our readers. Failure to do this may result in a score of 0 for untagged questions.

**You are responsible for ensuring your submission follows our requirements. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

After you have run the cell below and generated the zip file, you can open up your PDF <a href='hw02A.pdf' download>here</a> and download it. 

In [346]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

PandocMissing: Pandoc wasn't found.
Please check that pandoc is installed:
https://pandoc.org/installing.html